In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.0 MB/s eta 0:00:00


In [64]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn import metrics
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
import warnings
from sklearn.metrics import r2_score
from PIL import Image
import re

warnings.filterwarnings('ignore')
import catboost as cb
from catboost import CatBoostRegressor

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Using CLIP to create image-text embeddings, then classify each image to provided text category

In [59]:
from transformers import CLIPProcessor, CLIPModel

clipModel = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [89]:
#stores images and ids
image_list = []
image_homeids = []

In [87]:
#test categories to be used with CLIP model, for house quality scoring

# sizes = ["Small house", "Medium house", "Large house"]
styles = ["Newer house", "Older house"]
conditions = [
    "Well-maintained house",
    "House in need of minor repairs",
    "House in need of major renovations"
]

categories = [
    # f"{size}, {style}, {condition}"
    # for size in sizes
    f"{style}, {condition}"
    for style in styles
    for condition in conditions
]

for i, combination in enumerate(categories, 1):
    print(f"{i}. {combination}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
clipModel.to(device)
text_inputs = processor(text=categories, return_tensors="pt", padding=True)
text_inputs = {key: value.to(device) for key, value in text_inputs.items()}

1. Small house, Modern house, Newly constructed house
2. Small house, Modern house, Well-maintained house
3. Small house, Modern house, House in need of minor repairs
4. Small house, Modern house, House in need of major renovations
5. Small house, Traditional house, Newly constructed house
6. Small house, Traditional house, Well-maintained house
7. Small house, Traditional house, House in need of minor repairs
8. Small house, Traditional house, House in need of major renovations
9. Medium house, Modern house, Newly constructed house
10. Medium house, Modern house, Well-maintained house
11. Medium house, Modern house, House in need of minor repairs
12. Medium house, Modern house, House in need of major renovations
13. Medium house, Traditional house, Newly constructed house
14. Medium house, Traditional house, Well-maintained house
15. Medium house, Traditional house, House in need of minor repairs
16. Medium house, Traditional house, House in need of major renovations
17. Large house, 

In [90]:
count = 0
for anImg in os.listdir('/content/drive/MyDrive/!mlData/info558Kaggle/SimPictures'):
  imgHomeId = anImg[5:10]
  #remove trailing 0s
  imgHomeId = imgHomeId.lstrip('0')
  if imgHomeId not in image_homeids:
    image_list.append(Image.open(f'/content/drive/MyDrive/!mlData/info558Kaggle/SimPictures/{anImg}').convert("RGB"))
    image_homeids.append(imgHomeId)
    count += 1
print(count)


1200


In [91]:
#run clip model on text-image inputs to get text-image embeddings
inputs = processor(text=categories, images=image_list, return_tensors="pt", padding=True)
inputs = {key: value.to(device) for key, value in inputs.items()}
with torch.no_grad():
    outputs = clipModel(**inputs)
#get logies
logits_per_image = outputs.logits_per_image
#see category mapping
predicted_indices = torch.argmax(logits_per_image, dim=1)
predicted_categories = [categories[idx] for idx in predicted_indices]

In [99]:
clip_results_df = pd.DataFrame({
    "homeid": [int(homeid) for homeid in image_homeids],
    "predicted_category": predicted_categories
})
clip_results_df.head()

,homeid,predicted_category
0,5734,"Medium house, Modern house, House in need of m..."
1,4744,"Large house, Modern house, House in need of ma..."
2,7704,"Large house, Modern house, House in need of mi..."
3,6774,"Large house, Traditional house, House in need ..."
4,1626,"Medium house, Modern house, House in need of m..."


House reparis estimation text file classificaiton with zero-short transformers

In [151]:
from transformers import pipeline
bartClassifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device=device)

In [152]:
repair_degree = ['low repair costs', 'medium repair costs', 'high repair costs']

In [153]:
txtCount = 0
extracted_txt = []
txt_homeids = []
for aTxt in os.listdir('/content/drive/MyDrive/!mlData/info558Kaggle/SimReports'):
  aTxt_homeid = aTxt[5:10]
  aTxt_homeid = aTxt_homeid.lstrip('0')
  txtContent = open(f'/content/drive/MyDrive/!mlData/info558Kaggle/SimReports/{aTxt}', 'r').read()

  extracted_txt.append(txtContent)
  txt_homeids.append(aTxt_homeid)
  txtCount += 1
print(txtCount)

1200


In [154]:
bartResults = bartClassifier(extracted_txt, candidate_labels=repair_degree)

In [155]:
curCount =0
for text, result in zip(extracted_txt, bartResults):
    curCount +=1
    print(f"Text: {text}")
    print(f"Predicted category: {result['labels'][0]} with score: {result['scores'][0]:.2f}")
    print()
    if curCount == 5:
      break

Text: Inspection Report of Home ID # 05335:

This single-family property, located in Lothal, was built in 2024, featuring 6 bedrooms and 5.5 bathrooms, across a total living space of 1933 square feet. It is a 2-story construction and has been sold 1 time.

Key Findings from Inspection:

Upon entering the home, it is immediately apparent that significant neglect has led to various signs of deterioration and damage throughout the property. The interior paint has extensively peeled, and there are visible stains on many walls, suggesting water damage or mold issues. Flooring throughout the house, particularly in high-traffic areas, shows severe wear with numerous scratched, warped, or cracked sections. The kitchen and bathrooms exhibit outdated fixtures and considerable wear; cabinets and countertops are chipped and stained, and several plumbing fixtures are in disrepair, showing signs of leaks and rust. Electrical outlets and fixtures are outdated and not up to current safety standards. T

In [159]:
classified_repair_results = [result["labels"][0] for result in bartResults]
repair_results_df = pd.DataFrame({
    "homeid": [int(homeid) for homeid in txt_homeids],
    "predicted_category": classified_repair_results
})
repair_results_df.head()

,homeid,predicted_category
0,5335,high repair costs
1,6207,high repair costs
2,8393,high repair costs
3,5976,high repair costs
4,4552,high repair costs


Merging all generated feature dfs back to orig df

In [160]:
#load in original data first
data = pd.read_csv('/content/drive/MyDrive/!mlData/info558Kaggle/df-train_1000x12.csv')
test_data = pd.read_csv('/content/drive/MyDrive/!mlData/info558Kaggle/df-test_200x11.csv')
print(data.dtypes)
print(data.isnull().sum())
print(len(data))

homeid                int64
feat_yrbuilt          int64
feat_nrooms           int64
feat_nbed             int64
feat_nbath          float64
feat_sqft             int64
feat_proptype        object
feat_basement        object
feat_stories         object
feat_solar           object
feat_nsales           int64
target_homevalue      int64
dtype: object
homeid              0
feat_yrbuilt        0
feat_nrooms         0
feat_nbed           0
feat_nbath          0
feat_sqft           0
feat_proptype       0
feat_basement       0
feat_stories        0
feat_solar          0
feat_nsales         0
target_homevalue    0
dtype: int64
1000


In [161]:
data = data.merge(clip_results_df, on="homeid", how="left")
data = data.merge(repair_results_df, on="homeid", how="left")
print(data.dtypes)
data.head()

homeid                    int64
feat_yrbuilt              int64
feat_nrooms               int64
feat_nbed                 int64
feat_nbath              float64
feat_sqft                 int64
feat_proptype            object
feat_basement            object
feat_stories             object
feat_solar               object
feat_nsales               int64
target_homevalue          int64
predicted_category_x     object
predicted_category_y     object
dtype: object


,homeid,feat_yrbuilt,feat_nrooms,feat_nbed,feat_nbath,feat_sqft,feat_proptype,feat_basement,feat_stories,feat_solar,feat_nsales,target_homevalue,predicted_category_x,predicted_category_y
0,5936,2024,16,2,7.0,3456,Condominium,no,single,no,2,151512,"Large house, Modern house, House in need of mi...",high repair costs
1,7314,1899,13,2,1.0,500,Condominium,no,single,no,1,356344,"Large house, Traditional house, House in need ...",high repair costs
2,5448,2024,7,2,9.0,7414,SingleFam,no,single,no,2,78112,"Small house, Traditional house, Newly construc...",high repair costs
3,8560,2007,11,2,5.5,6619,SingleFam,no,single,no,3,872768,"Large house, Modern house, House in need of ma...",high repair costs
4,5783,2024,15,3,4.5,3226,MultiFam,no,multiple,no,1,946086,"Medium house, Modern house, Newly constructed ...",high repair costs


In [162]:
test_data = test_data.merge(clip_results_df, on="homeid", how="left")
test_data = test_data.merge(repair_results_df, on="homeid", how="left")
print(test_data.dtypes)
test_data.head()

homeid                    int64
feat_yrbuilt              int64
feat_nrooms               int64
feat_nbed                 int64
feat_nbath              float64
feat_sqft                 int64
feat_proptype            object
feat_basement            object
feat_stories             object
feat_solar               object
feat_nsales               int64
predicted_category_x     object
predicted_category_y     object
dtype: object


,homeid,feat_yrbuilt,feat_nrooms,feat_nbed,feat_nbath,feat_sqft,feat_proptype,feat_basement,feat_stories,feat_solar,feat_nsales,predicted_category_x,predicted_category_y
0,4997,1734,8,7,6.5,4500,MultiFam,yes,multiple,no,2,"Medium house, Modern house, House in need of m...",high repair costs
1,7647,2024,10,3,4.0,4470,Condominium,no,single,no,0,"Large house, Modern house, House in need of ma...",high repair costs
2,8202,1692,13,2,3.0,3985,SingleFam,no,single,no,2,"Small house, Traditional house, House in need ...",high repair costs
3,5573,2024,8,2,6.0,2079,SingleFam,no,single,no,3,"Small house, Modern house, Newly constructed h...",high repair costs
4,4612,2024,3,2,7.5,500,MultiFam,yes,multiple,no,2,"Medium house, Modern house, Newly constructed ...",high repair costs


In [163]:
#merge data with clip_results_df
dissimilar_rows = data[data["predicted_category_x"] != data["predicted_category_y"]]
if not dissimilar_rows.empty:
    print("Dissimilar rows found:")
    print(dissimilar_rows)
else:
    data = data.drop("predicted_category_y", axis=1)
    data = data.rename(columns={"predicted_category_x": "predicted_category"})
    print("No dissimilar rows found.")
print(data.dtypes)
data.head()

Dissimilar rows found:
     homeid  feat_yrbuilt  feat_nrooms  feat_nbed  feat_nbath  feat_sqft  \
0      5936          2024           16          2         7.0       3456   
1      7314          1899           13          2         1.0        500   
2      5448          2024            7          2         9.0       7414   
3      8560          2007           11          2         5.5       6619   
4      5783          2024           15          3         4.5       3226   
..      ...           ...          ...        ...         ...        ...   
995     466          1931           15          4         4.5       6187   
996    6367          1852            3          6         7.5       3779   
997    9678          1357            8          2         1.0        500   
998    6403          1397            3          2         1.0       6168   
999    9403          2024            4          3         1.0       5981   

    feat_proptype feat_basement feat_stories feat_solar  feat_ns

,homeid,feat_yrbuilt,feat_nrooms,feat_nbed,feat_nbath,feat_sqft,feat_proptype,feat_basement,feat_stories,feat_solar,feat_nsales,target_homevalue,predicted_category_x,predicted_category_y
0,5936,2024,16,2,7.0,3456,Condominium,no,single,no,2,151512,"Large house, Modern house, House in need of mi...",high repair costs
1,7314,1899,13,2,1.0,500,Condominium,no,single,no,1,356344,"Large house, Traditional house, House in need ...",high repair costs
2,5448,2024,7,2,9.0,7414,SingleFam,no,single,no,2,78112,"Small house, Traditional house, Newly construc...",high repair costs
3,8560,2007,11,2,5.5,6619,SingleFam,no,single,no,3,872768,"Large house, Modern house, House in need of ma...",high repair costs
4,5783,2024,15,3,4.5,3226,MultiFam,no,multiple,no,1,946086,"Medium house, Modern house, Newly constructed ...",high repair costs


In [164]:
def encode_cols(input_data):
  encoder = LabelEncoder()
  for col in input_data.columns:
      if input_data[col].dtype == 'object':
          input_data[col] = encoder.fit_transform(input_data[col])
  print(input_data.head())
  return input_data

In [165]:
data = encode_cols(data)

   homeid  feat_yrbuilt  feat_nrooms  feat_nbed  feat_nbath  feat_sqft  \
0    5936          2024           16          2         7.0       3456   
1    7314          1899           13          2         1.0        500   
2    5448          2024            7          2         9.0       7414   
3    8560          2007           11          2         5.5       6619   
4    5783          2024           15          3         4.5       3226   

   feat_proptype  feat_basement  feat_stories  feat_solar  feat_nsales  \
0              0              0             1           0            2   
1              0              0             1           0            1   
2              2              0             1           0            2   
3              2              0             1           0            3   
4              1              0             0           0            1   

   target_homevalue  predicted_category_x  predicted_category_y  
0            151512                     1   

In [166]:
X=data.drop(['target_homevalue','homeid'],axis=1)
y=data['target_homevalue']
print(X.head())
print(y.head())
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=412)

train_dataset = cb.Pool(X_train, y_train)
test_dataset = cb.Pool(X_test, y_test)

   feat_yrbuilt  feat_nrooms  feat_nbed  feat_nbath  feat_sqft  feat_proptype  \
0          2024           16          2         7.0       3456              0   
1          1899           13          2         1.0        500              0   
2          2024            7          2         9.0       7414              2   
3          2007           11          2         5.5       6619              2   
4          2024           15          3         4.5       3226              1   

   feat_basement  feat_stories  feat_solar  feat_nsales  predicted_category_x  \
0              0             1           0            2                     1   
1              0             1           0            1                     3   
2              0             1           0            2                    15   
3              0             1           0            3                     0   
4              0             0           0            1                     8   

   predicted_category_y  


Model training-CatBoost

In [167]:
catBoostModel = cb.CatBoostRegressor(loss_function="RMSE", task_type="GPU")

In [168]:
grid = {'iterations': [25, 50, 100, 150],
        'learning_rate': [0.001, 0.03, 0.1],
        'depth': [2, 4, 6, 8, 10],
        'l2_leaf_reg': [0.2, 0.5, 1, 3, 10, 15]}
catBoostModel.grid_search(grid, train_dataset)

Streaming output truncated to the last 5000 lines.
33:	learn: 378573.6621034	test: 373427.3325015	best: 373427.3325015 (33)	total: 1.88s	remaining: 883ms
34:	learn: 374477.6190311	test: 369091.4213817	best: 369091.4213817 (34)	total: 1.88s	remaining: 808ms
35:	learn: 370717.2408357	test: 365409.7764034	best: 365409.7764034 (35)	total: 1.89s	remaining: 736ms
36:	learn: 367193.7496984	test: 362100.9010671	best: 362100.9010671 (36)	total: 1.9s	remaining: 668ms
37:	learn: 363357.3495280	test: 359330.6317474	best: 359330.6317474 (37)	total: 1.91s	remaining: 604ms
38:	learn: 359807.4308071	test: 356921.2923881	best: 356921.2923881 (38)	total: 1.94s	remaining: 549ms
39:	learn: 356564.0013058	test: 353632.1583929	best: 353632.1583929 (39)	total: 1.95s	remaining: 489ms
40:	learn: 353296.7323574	test: 351257.4124918	best: 351257.4124918 (40)	total: 1.96s	remaining: 431ms
41:	learn: 350033.1352909	test: 348008.7954015	best: 348008.7954015 (41)	total: 1.97s	remaining: 375ms
42:	learn: 346999.26982

Training on fold [0/3]
0:	learn: 580190.5037637	test: 605772.1846718	best: 605772.1846718 (0)	total: 13.2ms	remaining: 1.97s
1:	learn: 544544.8927950	test: 569906.5316811	best: 569906.5316811 (1)	total: 24.8ms	remaining: 1.84s
2:	learn: 515869.4315507	test: 541630.7069391	best: 541630.7069391 (2)	total: 36.3ms	remaining: 1.78s
3:	learn: 487130.3068395	test: 511591.8173968	best: 511591.8173968 (3)	total: 67ms	remaining: 2.45s
4:	learn: 461860.1286626	test: 486975.6548143	best: 486975.6548143 (4)	total: 74.5ms	remaining: 2.16s
5:	learn: 439799.1895624	test: 465003.8294325	best: 465003.8294325 (5)	total: 94.2ms	remaining: 2.26s
6:	learn: 420027.5891171	test: 444696.9416631	best: 444696.9416631 (6)	total: 114ms	remaining: 2.34s
7:	learn: 402835.4625617	test: 427119.4593483	best: 427119.4593483 (7)	total: 127ms	remaining: 2.25s
8:	learn: 387091.2623536	test: 412463.1187417	best: 412463.1187417 (8)	total: 155ms	remaining: 2.42s
9:	learn: 373705.3470541	test: 401508.0550987	best: 401508.05509

{'params': {'depth': 6,
  'learning_rate': 0.1,
  'l2_leaf_reg': 3,
  'iterations': 150},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,

In [172]:
pred = catBoostModel.predict(X_test)
rmse = (np.sqrt(mean_squared_error(y_test, pred)))
r2 = r2_score(y_test, pred)
print("Testing performance")
print("RMSE: {:.2f}".format(rmse))
print("R2: {:.2f}".format(r2))

Testing performance
RMSE: 288946.70
R2: 0.59


In [173]:
def submit_csv(test_data, model):
  test_data_input = test_data.drop(['homeid'], axis=1)
  test_data_input = encode_cols(test_data_input)
  test_pred = model.predict(test_data_input)
  submission = pd.DataFrame({'homeid': test_data['homeid'], 'target_homevalue': test_pred})
  submission.to_csv('submission.csv', index=False)

In [174]:
submit_csv(test_data, catBoostModel)

   feat_yrbuilt  feat_nrooms  feat_nbed  feat_nbath  feat_sqft  feat_proptype  \
0          1734            8          7         6.5       4500              1   
1          2024           10          3         4.0       4470              0   
2          1692           13          2         3.0       3985              2   
3          2024            8          2         6.0       2079              2   
4          2024            3          2         7.5        500              1   

   feat_basement  feat_stories  feat_solar  feat_nsales  predicted_category_x  \
0              1             0           0            2                     7   
1              0             1           0            0                     0   
2              0             1           0            2                    15   
3              0             1           0            3                    14   
4              1             0           0            2                     9   

   predicted_category_y  


Linear Regression performance

In [132]:
def linear_reg_results(X_train, y_train, X_test, y_test):
  from sklearn.linear_model import LinearRegression
  from sklearn.metrics import mean_squared_error, r2_score
  reg = LinearRegression().fit(X_train, y_train)
  score = reg.score(X_train, y_train)
  y_pred = reg.predict(X_test)
  rmse = (np.sqrt(mean_squared_error(y_test, y_pred)))
  r2 = r2_score(y_test, y_pred)
  print("Testing performance")
  print("RMSE: {:.2f}".format(rmse))
  print("R2: {:.2f}".format(r2))

In [133]:
linear_reg_results(X_train, y_train, X_test, y_test)

Testing performance
RMSE: 421623.24
R2: 0.12
